In [13]:
from selenium import webdriver
from bs4 import BeautifulSoup # For HTML parsing
import requests # Website connections1
import pandas as pd 

In [14]:
def get_links(page_url):
    result = requests.get(page_url)
    soup = BeautifulSoup(result.content, 'lxml')
    a=[]
    for td in soup.find_all('td'):
        if td.a is not None:
            a.append(td.a)
    links=[]
    for link in a:
        links.append('https://markets.businessinsider.com' + link.get('href'))
    return links

In [15]:
links1 = get_links('https://markets.businessinsider.com/bonds/finder?borrower=71&maturity=shortterm&yield=&bondtype=2%2c3%2c4%2c16&coupon=&currency=184&rating=&country=19')
links1 = links1[4:]

In [16]:
links2 = get_links('https://markets.businessinsider.com/bonds/finder?borrower=71&maturity=midterm&yield=&bondtype=2%2c3%2c4%2c16&coupon=&currency=184&rating=&country=19')
links2 = links2[4:]

In [19]:
links = links1 + links2
len(links)

32

In [20]:
info=None
for i in range(len(links)):
    result = requests.get(links[i])
    soup = BeautifulSoup(result.content, 'lxml')
    table = soup.find_all('table', class_='table table-small no-margin-bottom')
    df = pd.read_html(str(table))[0]
    df=df.drop(9)
    df = df.set_index(0)
    array = df.loc[['ISIN','Issue Date','Maturity Date','Coupon'],][1]
    info = pd.concat([info,array],axis=1)

In [21]:
info = info.reset_index()

In [22]:
info

,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
0,ISIN,CA135087ZU15,CA135087J629,CA135087K296,CA135087J884,CA135087K601,CA135087TZ75,CA135087F585,CA135087UM44,CA135087YZ11,...,CA135087A610,CA135087B451,CA135087J397,CA135087F825,CA135087H490,CA135087UT96,CA135087D507,CA135087K528,CA135087E679,CA135087J546
1,Issue Date,8/2/2011,11/9/2018,5/6/2019,2/11/2019,11/4/2019,12/15/1990,4/11/2016,12/15/1991,9/8/2009,...,7/30/2012,7/2/2013,7/27/2018,8/3/2016,10/6/2017,8/17/1992,6/30/2014,10/11/2019,7/21/2015,10/5/2018
2,Maturity Date,6/1/2022,2/1/2021,8/1/2021,5/1/2021,2/1/2022,3/15/2021,9/1/2021,6/1/2022,6/1/2020,...,6/1/2023,6/1/2024,6/1/2029,6/1/2027,3/1/2023,6/1/2023,6/1/2025,3/1/2025,6/1/2026,3/1/2024
3,Coupon,2.750%,2.250%,1.500%,1.750%,1.500%,10.500%,0.750%,9.250%,3.500%,...,1.500%,2.500%,2.250%,1.000%,1.750%,8.000%,2.250%,1.250%,1.500%,2.250%


In [23]:
def get_historical_links(link):
    result = requests.get(link)
    soup = BeautifulSoup(result.content, 'lxml')
    for thing in soup.find_all('div', class_='col-xs-6 col-md-4 no-padding-left more-info-item'):
        if thing.a.text == 'Historical':
            return 'https://markets.businessinsider.com'+thing.a.get('href')+'/fse/2.1.2020_15.1.2020'

In [24]:
historicals=[]
for link in links:
    historicals.append(get_historical_links(link))

In [26]:
len(historicals)

32

In [29]:
historicals

['https://markets.businessinsider.com/bond/historical/2_750-canada-government-of-Bond-2022-ca135087zu15/fse/2.1.2020_15.1.2020',
 'https://markets.businessinsider.com/bond/historical/canadacd-bonds_201821-Bond-2021-ca135087j629/fse/2.1.2020_15.1.2020',
 'https://markets.businessinsider.com/bond/historical/canadacd-bonds_201921-Bond-2021-ca135087k296/fse/2.1.2020_15.1.2020',
 'https://markets.businessinsider.com/bond/historical/canadacd-bonds_201921-Bond-2021-ca135087j884/fse/2.1.2020_15.1.2020',
 'https://markets.businessinsider.com/bond/historical/canadacd-bonds_201922-Bond-2022-ca135087k601/fse/2.1.2020_15.1.2020',
 'https://markets.businessinsider.com/bond/historical/10_500-canada-government-of-Bond-2021-ca135087tz75/fse/2.1.2020_15.1.2020',
 'https://markets.businessinsider.com/bond/historical/canadacd-bonds_201621-Bond-2021-ca135087f585/fse/2.1.2020_15.1.2020',
 'https://markets.businessinsider.com/bond/historical/9_250-canada-government-of-Bond-2022-ca135087um44/fse/2.1.2020_15.1

In [30]:
browser=webdriver.Safari()

In [41]:
close_price = None
for historical in historicals:
    browser.get(historical)
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    table = soup.find_all('table',class_='table instruments')
    df = pd.read_html(str(table))[0]
    df = df.loc[:,[0,2]]
    df = df.sort_index(ascending=False)
    df = df.set_index(0)
    array = df[2]
    close_price = pd.concat([close_price,array],axis=1)

In [42]:
close_price = close_price.reset_index()

In [43]:
info.columns=[x for x in range(33)]

In [44]:
close_price.columns=[x for x in range(33)]

In [45]:
df = pd.concat([info,close_price])

In [46]:
df = df.set_index(0)

In [48]:
df.T.to_csv('a1_T.csv')

In [50]:
df.T

,ISIN,Issue Date,Maturity Date,Coupon,1/2/2020,1/3/2020,1/6/2020,1/7/2020,1/8/2020,1/9/2020,1/10/2020,1/13/2020,1/14/2020,1/15/2020
1,CA135087ZU15,8/2/2011,6/1/2022,2.750%,102.53,102.59,102.62,102.59,102.58,102.52,102.52,102.46,102.47,102.51
2,CA135087J629,11/9/2018,2/1/2021,2.250%,100.55,100.58,100.6,100.59,100.57,100.56,100.52,100.53,100.52,100.54
3,CA135087K296,5/6/2019,8/1/2021,1.500%,99.73,99.76,99.79,99.76,99.75,99.72,99.7,99.68,99.7,99.72
4,CA135087J884,2/11/2019,5/1/2021,1.750%,100.07,100.09,100.12,100.1,100.08,100.07,100.04,100.04,100.02,100.06
5,CA135087K601,11/4/2019,2/1/2022,1.500%,99.69,99.74,99.78,99.76,99.76,99.71,99.68,99.67,99.67,99.7
6,CA135087TZ75,12/15/1990,3/15/2021,10.500%,110.28,110.27,110.27,110.23,110.2,110.09,110.06,110.03,110.02,110.01
7,CA135087F585,4/11/2016,9/1/2021,0.750%,98.41,98.45,98.49,98.46,98.46,98.43,98.43,98.38,98.41,98.42
8,CA135087UM44,12/15/1991,6/1/2022,9.250%,117.73,117.79,117.82,117.77,117.74,117.63,117.56,117.52,117.51,117.54
9,CA135087YZ11,9/8/2009,6/1/2020,3.500%,100.7,100.7,100.69,100.69,100.68,100.66,100.65,100.65,100.64,100.64
10,CA135087E596,4/13/2015,9/1/2020,0.750%,99.26,99.28,99.28,99.27,99.28,99.28,99.28,99.27,99.28,99.3
